# 2. Poetry setup

## Preamble: Conda vs Poetry

**Conda** and **Poetry** are largely redundant:
  - Conda is primarily a environment manager (in fact not necessarily Python), but it can also manage packages.
  - Poetry is primarily a Python package manager (say, an upgrade of **pip**), but it can also create and manage Python environments (say, an upgrade of **Pyenv**).

Our strategy will be to compartmentalize their roles: let Conda be the environment manager and Poetry the package manager. The reason for this choice is that, as far as I understand:
  - Conda is best for managing Python environments and can be used for compiling and installing non-python packages, especially CUDA drivers (for GPU capability)
  - Poetry is more powerful than Conda as a Python package manager.

Therefore, in order to use both, we first create and activate a Conda environment (see previous notebook: *Conda setup*), and then we use Poetry ***nested within the Conda environment***, and use it solely as a package manager (we do not let it manage the Python environment).

## Prerequisites

**Conda:** First install Conda, and create and activate a Python environment called **NN**: see previous notebook *1. Conda setup*.

Select the kernel *NN* to run this notebook. Check:  
*(you should see a `*` for the NN environment)*

In [1]:
!conda info --envs

# conda environments:
#
fsdl-text-recognizer-2021     /home/brice/.conda/envs/fsdl-text-recognizer-2021
                         /home/brice/Dropbox/Info/Code/ML/NN/env
base                     /usr/local/anaconda3
NN                    *  /usr/local/anaconda3/envs/NN



Let's double check the version of Python is 3.9:

In [2]:
!python --version

Python 3.9.7


Although it's not necessary, you can check whether your system/environment is GPU capable:

In [3]:
!nvidia-smi

Tue Jan 25 15:28:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.94       Driver Version: 470.94       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P8    N/A /  N/A |      0MiB /  2004MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

We shall also need to open a terminal outside of this notebook, and make sure the Conda environment NN is activated in it:

`conda activate NN`

***For the remainder of this notebook, make sure your terminal is running the NN environment.***

## Install Poetry

(From https://python-poetry.org/docs/master/#installing-with-the-official-installer)  

Install Poetry **within the Conda environment NN**:  

In a terminal *outside this notebook*, ***with the NN environment activated (see above)***, run:

```
curl -sSL https://install.python-poetry.org | python -
```

Check your install:

In [10]:
!poetry --version

Poetry version 1.1.12


If for some reason you get a `ModuleNotFoundError`, this may fix your problem:  

```
pip install cleo tomlkit poetry.core requests cachecontrol cachy html5lib pkginfo virtualenv lockfile
```

## Project setup

Since we have already created our project (and a Python environment with Conda), we will not use `poetry new NN`, but instead:

*(as always make sure the Conda environment NN is activated)*

```
conda activate NN
poetry init
```

Call the package name 'NN', and press *Enter* multiple times to accept the all default suggestions, except the Python version: we want `3.9` (exactly 3.9) instead of `^3.9`(at least 3.9). *This is because, as far as I can tell, the installation of PyTorch fails for 3.10.*

This will create a **poetry.toml** file that should look like this:

```
[tool.poetry]
name = "NN"
version = "0.1.0"
description = ""
authors = ["seub <brice.loustau@gmail.com>"]

[tool.poetry.dependencies]
python = "3.9"

[tool.poetry.dev-dependencies]

[build-system]
requires = ["poetry-core>=1.0.0"]
build-backend = "poetry.core.masonry.api"
```

Next we can install dependencies:  

`poetry install`

This will create a **poetry.lock** file containing the exact versions of all packages installed for our project.

## Environment

We do not want to use Poetry as our Python environment manager, since we are using Conda for that. In particular, we will not use commands such as `poetry shell` or `source ` or `poetry run`.

Thankfully, thanks to the fact that we initialized Poetry within our Conda environment, Poetry is clever enough to realize it. Check this:

In [4]:
!poetry env info


Virtualenv
Python:         3.9.7
Implementation: CPython
Path:           /usr/local/anaconda3/envs/NN
Valid:          True

System
Platform: linux
OS:       posix
Python:   /usr/local/anaconda3/envs/NN


## Add packages

Let us add some new Python packages to our project using Poetry.  

Run in a terminal (with the NN environment activated as usual):

```
poetry add matplotlib
poetry add numpy
poetry add torch
```

We can also add packages only to the dev environment:

```
poetry add pytest --dev
```

We can check the list of installed packages and their dependencies:

In [5]:
!poetry show --tree

matplotlib 3.5.1 Python plotting package
├── cycler >=0.10
├── fonttools >=4.22.0
├── kiwisolver >=1.0.1
├── numpy >=1.17
├── packaging >=20.0
│   └── pyparsing >=2.0.2,<3.0.5 || >3.0.5 
├── pillow >=6.2.0
├── pyparsing >=2.2.1
├── python-dateutil >=2.7
│   └── six >=1.5 
└── setuptools-scm >=4
    ├── packaging >=20.0 
    │   └── pyparsing >=2.0.2,<3.0.5 || >3.0.5 
    └── tomli >=1.0.0 
numpy 1.22.1 NumPy is the fundamental package for array computing with Python.
pytest 6.2.5 pytest: simple powerful testing with Python
├── atomicwrites >=1.0
├── attrs >=19.2.0
├── colorama *
├── iniconfig *
├── packaging *
│   └── pyparsing >=2.0.2,<3.0.5 || >3.0.5 
├── pluggy >=0.12,<2.0
├── py >=1.8.2
└── toml *
torch 1.10.1 Tensors and Dynamic neural networks in Python with strong GPU acceleration
└── typing-extensions *


## The poetry.lock file

The **poetry.lock** file was created by `poetry install` and contains the exact versions of all packages installed.

As soon as this file is present, running `poetry install` resolves and installs all dependencies listed in **pyproject.toml**, but Poetry uses the exact versions listed in poetry.lock. As a result, they may not all be at the very latest available versions. This is by design. *(Source: https://python-poetry.org/docs/basic-usage/#using-poetry-run)*  

We should commit this file to version control (with **git**) to ensure that anyone who sets up the project uses the exact same versions of the dependencies.

If we want to update the dependencies to their latest versions, and consequently update the **poetry.lock** file, we can do:  

```
conda activate NN
poetry update
```

(This is equivalent to deleting the **poetry.lock** file and running `poetry install` again.)